In [1]:
# Import necessary libraries
import sqlite3
import pandas as pd

## 1. Load the xls files in the notebook as a dataframe


In [2]:
df = pd.read_excel('C:/Users/Sumit/Downloads/Rates_DF_Example.xls')

##  2. Perform data cleaning and any descriptive statistics as you see necessary

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10754 entries, 0 to 10753
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   service_code              10754 non-null  int64  
 1   service_code_description  10754 non-null  object 
 2   hospital_ID               10754 non-null  object 
 3   cost                      10754 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 336.2+ KB


In [4]:
df.head(10)

,service_code,service_code_description,hospital_ID,cost
0,29870,Knee surgery,1003820630,1432.34
1,29870,Knee surgery,1003820630,1016.51
2,29850,Hip Surgery,1003820630,1519.18
3,29850,Hip Surgery,1003820630,1519.18
4,22818,Covid Vaccine,1003820630,5402.73
5,22818,Covid Vaccine,1003820630,5402.73
6,29870,Knee surgery,1013970078,1432.34
7,29870,Knee surgery,1013970078,1016.51
8,29850,Hip Surgery,1013970078,1519.18
9,29850,Hip Surgery,1013970078,1519.18


In [5]:
# To check if there is any Null values
df.isna().sum()

service_code                0
service_code_description    0
hospital_ID                 0
cost                        0
dtype: int64

In [6]:
# Statistics of the Cost column Data
df['cost'].describe()

count    10754.000000
mean      1769.371624
std       1632.969857
min        351.220000
25%        701.460000
50%       1002.290000
75%       2501.200000
max      20843.360000
Name: cost, dtype: float64

## 3. Write a python function to calculate ranking of hospitals for each procedure code and identify costliest and cheapest hospital ID for each of the service code


In [8]:
# 
def calculate_hospital_ranking(df):
    # Find the index of the maximum cost for each service code
    max_cost_indices = df.groupby('service_code')['cost'].idxmax()
    
    # Get the corresponding rows using the indices
    costliest_hospitals = df.loc[max_cost_indices]
    
    # Group by service_code and rank hospitals based on cost
    df['rank'] = df.groupby('service_code')['cost'].rank(ascending=False)
    
    # find cheapest hospitals for each service_code
    cheapest_hospitals = df[df['rank'] == df.groupby('service_code')['rank'].transform('max')]
    
    return costliest_hospitals , cheapest_hospitals

# Identify costliest hospitals
costliest_hospitals , cheapest_hospitals = calculate_hospital_ranking(df)

# Print the results
print("Costliest Hospitals:")
print(costliest_hospitals[['service_code', 'hospital_ID', 'cost']])

print("Cheapest Hospitals:")
print(cheapest_hospitals[['service_code', 'hospital_ID', 'cost']])


Costliest Hospitals:
      service_code hospital_ID      cost
1546         22818  20-8624691  20843.36
1544         29850  20-8624691   6074.44
1542         29870  20-8624691   5691.52
Cheapest Hospitals:
      service_code hospital_ID     cost
1476         29870  20-8075502   351.22
1477         29870  20-8075502   351.22
1478         29850  20-8075502   470.02
1479         29850  20-8075502   470.02
6413         22818  27-1145142  1875.90


## 4. Create an SQlite database and load the xls file in to the SQlite database


In [9]:
# Path to the XLS file
xls_file_path = 'C:/Users/Sumit/Downloads/Rates_DF_Example.xls'

# Read the XLS file using pandas
xls_data = pd.read_excel(xls_file_path)

# Create a connection to the SQLite database 
db_connection = sqlite3.connect('example.db')

# Convert the data to a SQLite database table
xls_data.to_sql('rates_data', db_connection, if_exists='replace', index=False)

print("XLS data loaded into SQLite database successfully.")


XLS data loaded into SQLite database successfully.


## 5. Write an SQL query to calculate mean and standard deviation of the cost across each service code. 


In [10]:
import math
# Create a connection to the SQLite database
db_connection = sqlite3.connect('example.db')
cursor = db_connection.cursor()

# Calculate mean cost for each service code
mean_query = """
SELECT
    service_code,
    AVG(cost) AS mean_cost
FROM
    rates_data
GROUP BY
    service_code;
"""

cursor.execute(mean_query)
mean_results = cursor.fetchall()

# Calculate standard deviation for each service code
std_dev_query = """
SELECT
    service_code,
    cost
FROM
    rates_data;
"""

cursor.execute(std_dev_query)
std_dev_results = cursor.fetchall()

# Calculate mean squared differences
mean_squared_diff = {}
for service_code, cost in std_dev_results:
    if service_code not in mean_squared_diff:
        mean_squared_diff[service_code] = []
    mean_cost = next(item[1] for item in mean_results if item[0] == service_code)
    mean_squared_diff[service_code].append((cost - mean_cost) ** 2)

# Calculate standard deviation
std_deviation = {}
for service_code, squared_diff_list in mean_squared_diff.items():
    mean_squared_diff_sum = sum(squared_diff_list)
    std_deviation[service_code] = math.sqrt(mean_squared_diff_sum / len(squared_diff_list))

# Print mean and standard deviation for each service code
for item in mean_results:
    service_code = item[0]
    mean_cost = item[1]
    std_dev_cost = std_deviation.get(service_code, 0)  # If service code has no standard deviation data, use 0
    print(f"Service Code: {service_code}, Mean Cost: {mean_cost}, Standard Deviation: {std_dev_cost}")

# Close the connection
db_connection.close()


Service Code: 22818, Mean Cost: 3510.2164387385487, Standard Deviation: 1717.3378553213065
Service Code: 29850, Mean Cost: 971.4395021097149, Standard Deviation: 493.6001521488956
Service Code: 29870, Mean Cost: 828.8853733406972, Standard Deviation: 504.4764862143422
